In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model


In [2]:
model = load_model('chatbot_model.h5')

In [3]:
import json
import random
intents = json.loads(open('commands.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))


In [4]:
def clean_up_sentence(sentence):
    # tokenize 
    sentence_words = nltk.word_tokenize(sentence)
    # stemmming
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


In [5]:
# return bag of words array

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))


In [6]:
def predict_class(sentence, model):
    # filter out predictions based on below threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


In [7]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result


In [8]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res


In [9]:
!pip install SpeechRecognition
!pip install pipwin
!pipwin install pyaudio


Package `pyaudio` found in cache
https://download.lfd.uci.edu/pythonlibs/x6hvwk7i/PyAudio-0.2.11-cp38-cp38-win_amd64.whl
PyAudio-0.2.11-cp38-cp38-win_amd64.whl
[*] 0 bytes / 111 kB @ 0 bytes/s [------------------] [0.0%, 0s left]   [*] 0 bytes / 111 kB @ 0 bytes/s [------------------] [0.0%, 0s left]   [*] 0 bytes / 111 kB @ 0 bytes/s [------------------] [0.0%, 0s left]   [*] 0 bytes / 111 kB @ 0 bytes/s [------------------] [0.0%, 0s left]   [*] 0 bytes / 111 kB @ 0 bytes/s [------------------] [0.0%, 0s left]   [*] 0 bytes / 111 kB @ 0 bytes/s [------------------] [0.0%, 0s left]   [*] 0 bytes / 111 kB @ 0 bytes/s [------------------] [0.0%, 0s left]   [*] 0 bytes / 111 kB @ 0 bytes/s [------------------] [0.0%, 0s left]   [*] 0 bytes / 111 kB @ 0 bytes/s [------------------] [0.0%, 0s left]   [*] 0 bytes / 111 kB @ 0 bytes/s [------------------] [0.0%, 0s left]   [*] 0 bytes / 111 kB @ 0 bytes/s [------------------] [0.0%, 0s left]   [*] 0 bytes / 111 kB @ 0 bytes/s [--------------

In [10]:
#take Speech Inputs
while(1):
    import speech_recognition as sr
    r = sr.Recognizer()
    audio = ''
    with sr.Microphone() as source:
        print("Speak...")
        audio = r.listen(source, phrase_time_limit=5)
    print("Stop.")
    try:
        text = r.recognize_google(audio,language='en-US')
        print("You : ", text)
        chatbot_response(text)
        res = chatbot_response(text)
        print("Bot: " + res + '\n\n')
    except:
        print("Could not understand your audio, PLease try again!")


Speak...
Stop.
You :  hai
Bot: Hello, thanks for asking


Speak...
Stop.
You :  how are you
Bot: Hello, thanks for asking


Speak...
Stop.
You :  what is your age
Bot: I can guide you through Adverse drug reaction list, Blood pressure tracking, Hospitals and Pharmacies


Speak...
Stop.
Could not understand your audio, PLease try again!
Speak...
Stop.
You :  how many pounds is how many pharmacies near me
Bot: Please provide pharmacy name


Speak...
Stop.
Could not understand your audio, PLease try again!
Speak...
Stop.
You :  please provide the list of pharmacy
Bot: Please provide pharmacy name


Speak...
Stop.
Could not understand your audio, PLease try again!
Speak...
Stop.
You :  provide the list of hospitals
Bot: Please provide hospital name or location


Speak...
Stop.
Could not understand your audio, PLease try again!
Speak...
Stop.
Could not understand your audio, PLease try again!
Speak...
Stop.
You :  provide the list of hospitals in Madurai
Bot: Please provide hospital name or

KeyboardInterrupt: 